In [ ]:
!pip install tensorflowjs
!pip install --upgrade tensorflowjs
!import tensorflow as tf

In [125]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [126]:
# Membaca dataset dari file CSV
data = pd.read_csv('/content/tes.csv')
print(data.head(5))

   foodId                 Nama         Tipe Tipe 1  Tipe 2
0       1          Sosis Bakar  ayam-daging   ayam  daging
1       2  Ngohiong Ayam Udang  ayam-daging   ayam  daging
2       3           Rawon Ayam  ayam-daging   ayam  daging
3       4   Usus Goreng Crispy  ayam-daging   ayam  daging
4       5      Ceker Rica Rica  ayam-daging   ayam  daging


In [ ]:
# Mengubah kolom tipe menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
data['tipe_encoded'] = label_encoder.fit_transform(data['Tipe'])

print(data)

In [128]:
# Melakukan preprocessing pada dataset
onehot_encoder = OneHotEncoder(sparse=False)
tipe_encoded = data['tipe_encoded'].values.reshape(-1, 1)
tipe_encoded = onehot_encoder.fit_transform(tipe_encoded)
tipe_encoded_df = pd.DataFrame(tipe_encoded, columns=['tipe_' + str(i) for i in range(tipe_encoded.shape[1])])
data = pd.concat([data, tipe_encoded_df], axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [129]:
# Memisahkan fitur dan label
X = data.drop(['foodId', 'Nama', 'Tipe', 'Tipe 1', 'Tipe 2', 'tipe_encoded'], axis=1)
y = data['tipe_encoded']

In [130]:
# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [131]:
# Membangun model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.nunique(), activation='softmax'))  # Jumlah kelas sesuai dengan jumlah tipe makanan yang unik

In [132]:
# Mengompilasi model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [133]:
# Melatih model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
32/32 [==============================] - 1s 2ms/step - loss: 2.3208 - accuracy: 0.5747
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 1.9254 - accuracy: 0.9037
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 1.3368 - accuracy: 0.9037
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7329 - accuracy: 0.9194
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.3719 - accuracy: 0.9548
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2104 - accuracy: 0.9754
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1337 - accuracy: 0.9941
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0906 - accuracy: 0.9941
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0642 - accuracy: 0.9941
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0472 - accuracy: 0.9941


In [134]:
# Evaluasi model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Akurasi model:', accuracy)

Akurasi model: 0.9921568632125854


In [135]:
def recommend_food(model, data, food_id, food_name, num_recommendations=10):
    # Find the row in the data based on the food_id
    food_row = data[data['foodId'] == food_id]
    
    # Create a DataFrame with the input features
    input_features = pd.DataFrame({
        'foodId': [food_id],
        'Nama': [food_name],
        'Tipe': [food_row['Tipe'].values[0]]
    })
    
    # Preprocess the input features
    input_features['tipe_encoded'] = label_encoder.transform(input_features['Tipe'])
    input_features_encoded = onehot_encoder.transform(input_features['tipe_encoded'].values.reshape(-1, 1))
    
    # Predict the food type
    prediction = model.predict(input_features_encoded)
    
    # Decode the predicted food type
    predicted_food_type = label_encoder.inverse_transform([prediction.argmax()])[0]
    
    # Get recommended foods of the predicted type
    recommended_foods = data[data['Tipe'] == predicted_food_type]['Nama'].tolist()
    
    # Remove the input food from the recommended list
    recommended_foods = [food for food in recommended_foods if food != food_name]
    
    # Return the top recommendations
    top_recommendations = recommended_foods[:num_recommendations]
    
    return top_recommendations



In [136]:
food_id = 280
food_name = 'Sate Kerang'

top_recommendations = recommend_food(model, data, food_id, food_name, num_recommendations=10)
print("Top 10 Recommended foods: ")
for i, food in enumerate(top_recommendations, 1):
    print(f"{i}. {food}")

1/1 [==============================] - 0s 59ms/step
Top 10 Recommended foods: 
1. Ikan Mas Pindang
2. Ikan Mujair Goreng
3. Ikan Bandeng Asam Manis
4. Ikan Mas Bakar
5. Ikan Mas Pesmol
6. Ikan Bandeng Goreng
7. Ikan Bandeng Pepes
8. Ikan Mas Goreng
9. Ikan Bandeng Gulai
10. Ikan Kembung Pesmol


In [137]:
saved_model_path = "./my_model.h5"
model.save(saved_model_path)

In [138]:
! tensorflowjs_converter \
    --input_format=keras \
    {saved_model_path} \
    "./"

2023-06-05 10:00:27.390694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
